In [2]:
import requests
import pandas as pd
import sqlite3
import lxml.html

### scraping data from html -1

In [3]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Nintendo_Entertainment_System_games")

In [5]:
r #response object is called

<Response [200]>

###### typing r.text and running it will give the text of source code which we saw by typing ^u 

In [6]:
qaz = lxml.html.fromstring(r.text) #lxml is used to parse html content.

In [7]:
qaz

<Element html at 0x7fce7a77b598>

help(qaz)

we want the table element which is in qaz 

In [8]:
soft_list_table = qaz.get_element_by_id("softwarelist")

In [9]:
soft_list_table

<Element table at 0x7fce7a7c3ae8>

In [10]:
soft_list_table.getchildren() # we get the output as list

[<Element tbody at 0x7fce7a77bd68>]

soft_list_table.getchildren()[0] in this case gives same result as above since the table has only one element within it
which is the tbody(as in source code).

soft_list_table.getchildren()[0].getchildren() #will show all the rows within tbody (table body)

In [11]:
tablebody=soft_list_table.getchildren()[0] 

In [12]:
len(tablebody) #716 row elements in the table, 2 are header <th> and 714 are table data <td> each having 
#+ 5 values.

716

In [13]:
tablebody.getchildren()[0].text_content() # 1st row, within which we have 4 elements

'\nTitle[3][4]\n\nRelease date\n\nPublisher(s)[3][4][5]\n\nDeveloper(s)\n'

In [14]:
tablebody.getchildren()[0].getchildren() # this can be checked in the source code, it produces tag of header i.e <th>
#+ title,release date, publishers and developers.

[<Element th at 0x7fce7a7941d8>,
 <Element th at 0x7fce7a794048>,
 <Element th at 0x7fce7a794228>,
 <Element th at 0x7fce7a794278>]

In [15]:
tablebody.getchildren()[1].getchildren() # 2 tags of header again as it mentions th-table header, 
#+ cross checking with source code it is North America pal regions

[<Element th at 0x7fce7a7943b8>, <Element th at 0x7fce7a794098>]

In [16]:
tablebody.getchildren()[2].getchildren() #third row which has 5 elements, where td is table data 

[<Element td at 0x7fce7a794318>,
 <Element td at 0x7fce7a794188>,
 <Element td at 0x7fce7a794458>,
 <Element td at 0x7fce7a794548>,
 <Element td at 0x7fce7a794598>]

In [17]:
print(tablebody.getchildren()[2].getchildren()[0].text_content())# inside tablebody, its 3rd row element's 1st child is  
#+ the hyperlink for '10-yard fight'
print(tablebody.getchildren()[2].getchildren()[1].text_content()) # 3rd row, 2nd element
print(tablebody.getchildren()[2].getchildren()[2].text_content()) # 3rd row, 3rd element
print(tablebody.getchildren()[2].getchildren()[3].text_content()) # 3rd row, 4th element
print(tablebody.getchildren()[2].getchildren()[4].text_content().strip()) # 3rd row, 5th element

10-Yard Fight
October 18, 1985
December 6, 1986
Nintendo
Irem


https://www.youtube.com/watch?v=u0OeZfIfBRI intro to html

https://www.youtube.com/watch?v=wvR40su_XBM intro to html tables

### parsing with lxml into pandas -2

In [18]:
for ro in tablebody.getchildren()[2]:
    print (ro.text_content().strip()) # specifically callig for second row from where actual values start.

10-Yard Fight
October 18, 1985
December 6, 1986
Nintendo
Irem


In [19]:
data_sample = {
    "title"     :[],
    "NA"        :[],
    "PAL"       :[],
    "publishers":[],
    "developers":[] 
}
for row in tablebody.getchildren()[2:5]:
    data_sample["title"].append(row.getchildren()[0].text_content().strip())
    data_sample["NA"].append(row.getchildren()[1].text_content().strip())
    data_sample["PAL"].append(row.getchildren()[2].text_content().strip())
    data_sample["publishers"].append(row.getchildren()[3].text_content().strip())
    data_sample["developers"].append(row.getchildren()[4].text_content().strip())

In [126]:
data_sample

{'title': ['10-Yard Fight', '1942', '1943: The Battle of Midway'],
 'NA': ['October 18, 1985', 'November 1986', 'October 1988'],
 'PAL': ['December 6, 1986', 'Unreleased', 'Unreleased'],
 'publishers': ['Nintendo', 'Capcom', 'Capcom'],
 'developers': ['Irem', 'Micronics', 'Capcom']}

In [21]:
pd.DataFrame(data_sample)

,title,NA,PAL,publishers,developers
0,10-Yard Fight,"October 18, 1985","December 6, 1986",Nintendo,Irem
1,1942,November 1986,Unreleased,Capcom,Micronics
2,1943: The Battle of Midway,October 1988,Unreleased,Capcom,Capcom


In [22]:
data = {
    "title"     :[],
    "NA"        :[],
    "PAL"       :[],
    "publishers":[],
    "developers":[] 
}
for row in tablebody.getchildren()[2:]:
    data["title"].append(row.getchildren()[0].text_content().strip())
    data["NA"].append(row.getchildren()[1].text_content().strip())
    data["PAL"].append(row.getchildren()[2].text_content().strip())
    data["publishers"].append(row.getchildren()[3].text_content().strip())
    data["developers"].append(row.getchildren()[4].text_content().strip())

IndexError: list index out of range

tablebody_getchildren() = is the 714 rows starting from index number 2

ERROR: some of the developer column values are empty in the table

In [23]:
data = {
    "title"     :[],
    "NA"        :[],
    "PAL"       :[],
    "publishers":[],
    "developers":[] 
}

for row in tablebody.getchildren()[2:]:
  
    data["title"].append(row.getchildren()[0].text_content().strip())
    data["NA"].append(row.getchildren()[1].text_content().strip())
    data["PAL"].append(row.getchildren()[2].text_content().strip())
    data["publishers"].append(row.getchildren()[3].text_content().strip())
    
    if len(row) < 5:
        data["developers"].append("Not_Available") #checked in wiki page for any existing word Not_Available
    else:
        data["developers"].append(row.getchildren()[4].text_content().strip())
    
df = pd.DataFrame(data)

In [24]:
df

,title,NA,PAL,publishers,developers
0,10-Yard Fight,"October 18, 1985","December 6, 1986",Nintendo,Irem
1,1942,November 1986,Unreleased,Capcom,Micronics
2,1943: The Battle of Midway,October 1988,Unreleased,Capcom,Capcom
3,The 3-D Battles of WorldRunner,September 1987,Unreleased,Acclaim Entertainment,Square
4,720°,November 1989,Unreleased,Mindscape,Tengen
5,8 Eyes,January 1990,Unreleased,Taxan,Thinking Rabbit
6,Abadox,March 1990,Unreleased,Milton Bradley Company,Natsume
7,The Addams Family,January 1992,December 1992,Ocean Software,Ocean Software
8,The Addams Family: Pugsley's Scavenger Hunt,August 1993,1992,Ocean Software,Ocean Software
9,Advanced Dungeons & Dragons: DragonStrike,July 1992,Unreleased,FCI,Westwood Studios


In [27]:
df[df.developers.isin(["Not_Available"])] # alternatively can use df[df.developers == "Empty"]

,title,NA,PAL,publishers,developers
114,Castle of Dragon,June 1990,Unreleased,SETA,Not_Available
130,Classic Concentration,September 1990,Unreleased,GameTek,Not_Available
211,Family Feud,May 1991,Unreleased,GameTek,Not_Available
491,Remote Control,May 1990,Unreleased,Hi Tech Expressions,Not_Available
525,Sesame Street: Countdown,February 1992,Unreleased,Hi Tech Expressions,Not_Available
583,Super Jeopardy!,September 1991,Unreleased,GameTek,Not_Available
623,Thunderbirds,September 1990,Unreleased,Activision,Not_Available
633,Tom and Jerry,December 1991,"October 22, 1992",Hi Tech Expressions,Not_Available
659,Uncharted Waters,November 1991,Unreleased,Koei,Not_Available
661,The Untouchables,January 1991,Unreleased,Ocean Software,Not_Available


In [28]:
df[df.title.isin(["Advanced Dungeons & Dragons: Hillsfar","Baseball Stars 2"])] # the developers for these values 
#+ is empty even after looping and does not cause error.

,title,NA,PAL,publishers,developers
11,Advanced Dungeons & Dragons: Hillsfar,February 1993,Unreleased,FCI,
62,Baseball Stars 2,July 1992,Unreleased,Romstar,


In [132]:
df.developers[11]

''

### Inserting pandas data into sqlite -3

In [89]:
df_db = df[df.developers.isin(["Not_Available","Nintendo Research & Development 1","TOSE"])]
#we will be loading this DF in sql DB.

In [94]:
df_db.head(5)#sample view

,title,NA,PAL,publishers,developers
51,Balloon Fight,August 1986,"December 15, 1986",Nintendo,Nintendo Research & Development 1
53,Bandai Golf: Challenge Pebble Beach,February 1989,Unreleased,Bandai,TOSE
57,Barker Bill's Trick Shooting,August 1990,"June 27, 1991",Nintendo,Nintendo Research & Development 1
59,Baseball,"October 18, 1985","September 1, 1986",Nintendo,Nintendo Research & Development 1
63,Bases Loaded,July 1988,Unreleased,Jaleco,TOSE


##### tuple object supports sqlite interfacing when using INSERT. 

In [134]:
for q in df_db.itertuples():
    print(q[1:])  
    #pass

('Balloon Fight', 'August 1986', 'December 15, 1986', 'Nintendo', 'Nintendo Research & Development 1')
('Bandai Golf: Challenge Pebble Beach', 'February 1989', 'Unreleased', 'Bandai', 'TOSE')
("Barker Bill's Trick Shooting", 'August 1990', 'June 27, 1991', 'Nintendo', 'Nintendo Research & Development 1')
('Baseball', 'October 18, 1985', 'September 1, 1986', 'Nintendo', 'Nintendo Research & Development 1')
('Bases Loaded', 'July 1988', 'Unreleased', 'Jaleco', 'TOSE')
('Bases Loaded II: Second Season', 'January 1990', 'Unreleased', 'Jaleco', 'TOSE')
('Bases Loaded 3', 'September 1991', 'Unreleased', 'Jaleco', 'TOSE')
('Bases Loaded 4', 'April 1993', 'Unreleased', 'Jaleco', 'TOSE')
('Castle of Dragon', 'June 1990', 'Unreleased', 'SETA', 'Not_Available')
('Chubby Cherub', 'October 29, 1986', 'Unreleased', 'Bandai', 'TOSE')
('Classic Concentration', 'September 1990', 'Unreleased', 'GameTek', 'Not_Available')
('Clu Clu Land', 'October 18, 1985', 'February 15, 1987', 'Nintendo', 'Nintendo Res

In [122]:
#identation are very important for proper execution.
conn = sqlite3.connect(":memory:") # database(stored in variable conn) is created temprorarily in memory
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE GAMES(title, NA, PAL, publishers, developers)
""")
for row in df_db.itertuples():
    insert_table = """
    INSERT INTO GAMES(title, NA, PAL, publishers, developers) \
    VALUES (?,?,?,?,?)
"""
    cursor.execute(insert_table, row[1:])
#conn.close()
conn.commit()

#### Retrieving data from table GAMES

In [123]:
cursor.execute("""
    SELECT *
    FROM GAMES
    WHERE PAL = "Unreleased"
""")  

In [124]:
for row in cursor.execute("""
    SELECT *
    FROM GAMES
    WHERE PAL = "Unreleased"
"""):
    print(row)

('Bandai Golf: Challenge Pebble Beach', 'February 1989', 'Unreleased', 'Bandai', 'TOSE')
('Bases Loaded', 'July 1988', 'Unreleased', 'Jaleco', 'TOSE')
('Bases Loaded II: Second Season', 'January 1990', 'Unreleased', 'Jaleco', 'TOSE')
('Bases Loaded 3', 'September 1991', 'Unreleased', 'Jaleco', 'TOSE')
('Bases Loaded 4', 'April 1993', 'Unreleased', 'Jaleco', 'TOSE')
('Castle of Dragon', 'June 1990', 'Unreleased', 'SETA', 'Not_Available')
('Chubby Cherub', 'October 29, 1986', 'Unreleased', 'Bandai', 'TOSE')
('Classic Concentration', 'September 1990', 'Unreleased', 'GameTek', 'Not_Available')
('Demon Sword', 'January 1990', 'Unreleased', 'Taito', 'TOSE')
("Dusty Diamond's All-Star Softball", 'July 1990', 'Unreleased', 'Brøderbund', 'TOSE')
('Family Feud', 'May 1991', 'Unreleased', 'GameTek', 'Not_Available')
('Frankenstein: The Monster Returns', 'July 1991', 'Unreleased', 'Bandai', 'TOSE')
('Golf Grand Slam', 'November 1991', 'Unreleased', 'Atlus', 'TOSE')
('Kid Niki: Radical Ninja', 'Nov

In [125]:
data = cursor.execute("""SELECT * 
                      FROM GAMES 
                      WHERE developers= "Not_Available"
""")
for row in data:
   print("title = ", row[0])
   print("NA = ", row[1])
   print("PAL = ", row[2])
   print("publishers = ", row[3])
   print("developers= ", row[4], "\n")
conn.close()

title =  Castle of Dragon
NA =  June 1990
PAL =  Unreleased
publishers =  SETA
developers=  Not_Available 

title =  Classic Concentration
NA =  September 1990
PAL =  Unreleased
publishers =  GameTek
developers=  Not_Available 

title =  Family Feud
NA =  May 1991
PAL =  Unreleased
publishers =  GameTek
developers=  Not_Available 

title =  Remote Control
NA =  May 1990
PAL =  Unreleased
publishers =  Hi Tech Expressions
developers=  Not_Available 

title =  Sesame Street: Countdown
NA =  February 1992
PAL =  Unreleased
publishers =  Hi Tech Expressions
developers=  Not_Available 

title =  Super Jeopardy!
NA =  September 1991
PAL =  Unreleased
publishers =  GameTek
developers=  Not_Available 

title =  Thunderbirds
NA =  September 1990
PAL =  Unreleased
publishers =  Activision
developers=  Not_Available 

title =  Tom and Jerry
NA =  December 1991
PAL =  October 22, 1992
publishers =  Hi Tech Expressions
developers=  Not_Available 

title =  Uncharted Waters
NA =  November 1991
PAL =

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html - sqlalchamy-DF to Sql


https://stackoverflow.com/questions/6318126/why-do-you-need-to-create-a-cursor-when-querying-a-sqlite-database - why we
need cursor
